#  Shadow Models

The adversary trains $k$ shadow models $f^i_{shadow}()$, each on a dataset (shadow data) that is similar
in format and distribution as the __target model's private training set__.

Generate shadow data drawing samples from the population where the target model's training data
are drawn from. The shadow model must be __trained in a similar fashion as the target model__.
The larger the number of shadow models, the better the accuracy of the attack model.

In [1]:
from mblearn import ShadowModels

from sklearn.ensemble import RandomForestClassifier

import pandas as pd

We are going to use the synthetic data generated in `data_synthesis_playground.ipynb`

In [2]:
synth_data = pd.read_csv('synthetic_irir.csv')

In [3]:
print(synth_data.shape)
synth_data.head()

(150, 5)


,0,1,2,3,label
0,0.495151,0.471551,0.113910,0.047197,0
1,0.083423,0.210501,0.006217,0.152407,0
2,0.694754,0.832320,0.222306,0.176190,0
3,0.620594,0.812440,0.244068,0.134671,0
4,0.625834,0.568340,0.177182,0.100158,0


Since we know which learner is used in the target model we are going to use the same:

In [4]:
shadow_learner =  RandomForestClassifier(n_estimators=100)

In [5]:
x = synth_data.iloc[:, :-1].to_numpy()
y = synth_data['label'].to_numpy()

In [6]:
sh = ShadowModels(x, y, n_models=5, target_classes=3, learner=shadow_learner)

using sklearn shadow models


In [7]:
sh

Shadow models: 5, <class 'sklearn.ensemble.forest.RandomForestClassifier'>
lengths of data splits : [30, 30, 30, 30, 30]

The shadow models results are in `results` attribute. The array structure is the following:

+ Last column `[:, -1]` specifies if the record was in a training (1) or test (0) set of a shadow model
+ Second last column `[:, -2]` is the label of the synthetic data.
+ The remanining columns are the predicted class probability vector of the record satisfying: `len(result[:, :-2]) == target_classes`

In [8]:
sh.results[:5]

array([[0.  , 0.1 , 0.9 , 2.  , 1.  ],
       [0.92, 0.08, 0.  , 0.  , 1.  ],
       [0.02, 0.98, 0.  , 1.  , 1.  ],
       [0.17, 0.81, 0.02, 1.  , 1.  ],
       [0.  , 1.  , 0.  , 1.  , 1.  ]])